In [1]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
import os
# -----------------------------------------------------------------------------
# openai and weaviate
import openai
openai.api_key = os.getenv("OPENA_AI_KEY")
import weaviate  
WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_APIKEY = os.getenv("WEAVIATE_API_KEY")
from weaviate.auth import AuthApiKey
# -----------------------------------------------------------------------------
# import utilities
from rag_utils import (
    load_data_to_sql_db, 
    text_to_query_engine, 
    build_sentence_window_index_vector_DB,
    setup_query_engines,
    get_retry_guideline_response,
    evaluate_and_transform_query,
    build_sentence_window_index)
# from RAG.WO_notebooks.rag_utils import build_sentence_window_index_vector_DB


# -----------------------------------------------------------------------------
# llama_index imports
#--------------------------------------------------------------------------------
from llama_index.response.notebook_utils import display_response
from llama_index import (
    OpenAIEmbedding, 
    Document,)
from llama_index.llms import OpenAI
from llama_index.vector_stores import WeaviateVectorStore


/Users/hamidadesokan/miniforge3/envs/wo_rag/lib/python3.10/site-packages/llama_index/download/module.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
model_name = "gpt-3.5-turbo"
embedding_model_name = "text-embedding-3-large"
# embedding_model_name="local:BAAI/bge-small-en-v1.5"
llm = OpenAI(temperature=0.1, model = model_name)
embed_model = OpenAIEmbedding(model= embedding_model_name)

In [3]:
auth_config = AuthApiKey(api_key=WEAVIATE_APIKEY)

client = weaviate.Client(
  url=WEAVIATE_URL,
  auth_client_secret=auth_config)

/Users/hamidadesokan/miniforge3/envs/wo_rag/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [3]:
# # Connect to a WCS instance
# client = weaviate.connect_to_wcs(
#     cluster_url=WEAVIATE_URL,
#     auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_APIKEY))

# Load Work Order Data

In [4]:
# path to the raw dataset
table_name = "work_order_table"
dbpath = "./data/wo_data.db"
wo_data_path="/Users/hamidadesokan/Dropbox/2_Skill_Development/DLML/genai_applications/embeddings/RAG/WO_notebooks/data/excavator_2015_cleaned_forpdl.csv"
conn, engine, data = load_data_to_sql_db(wo_data_path, dbpath, table_name)

# test the query engine

In [6]:
# model_name = "gpt-3.5-turbo"
# embedding_model_name = "text-embedding-3-large"
# llm = OpenAI(temperature=0.1, model = model_name)
# embed_model = OpenAIEmbedding(model= embedding_model_name)
# query_engine,sql_database, service_context = text_to_query_engine(model_name, embedding_model_name, table_name, engine)
# # query_str = "Which work oder cost 183.05?"
# query_str = "How much in total did we spend in 2011?"
# response = query_engine.query(query_str)
# display_response(response)

**`Final Response:`** In 2011, we spent a total of $1,534,557.32.

# Create document object

In [5]:
columns_to_embed = ["OriginalShorttext", 
                "MajorSystem" , "Part",
                "Action", "FM", "Location", "FuncLocation"]

columns_to_metadata = ['Asset',  'Cost', 'RunningTime', 'Variant', 'Comments', 'SuspSugg', 'Rule']


docs = []
for i, row in data.iterrows():
    to_metadata = {col: row[col] for col in columns_to_metadata if col in row}
    values_to_embed = {k: str(row[k]) for k in columns_to_embed if k in row}
    to_embed = '\n'.join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
    newDoc = Document(text=to_embed, metadata=to_metadata)
    docs.append(newDoc)

# Create a single document from a list of Documents
# this is what we will chunk up and store with its embedding in Weaviate
document = Document(text="\n\n".join([doc.text for doc in docs]))

# Build Sentence window

In [6]:
# load work_order_schema from json
client.schema.delete_class("WorkOrder")
import json
with open("./config/work_order_schema", "r") as f:
    work_order_schema = json.load(f)

client.schema.create(work_order_schema)
print("Product schema was created.")


Product schema was created.


In [8]:
work_order_index = build_sentence_window_index_vector_DB(
    document = [document],
    client = client,
    llm = llm,
    embed_model=embed_model,
    prefix = "Work_order_sent_win_index"
)

/Users/hamidadesokan/miniforge3/envs/wo_rag/lib/python3.10/site-packages/weaviate/__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 8260 tokens (8260 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

# Save the index in vectore store

In [9]:
query_router_engine = setup_query_engines(sql_database, work_order_index, table_name)
response = get_retry_guideline_response(query_router_engine, "How much in total did we spend in 2011?", guideline = False)
print(response)

In [10]:
query_router_engine

In [11]:
# query_str = "Which work oder cost 183.05?"
query_str = "How much in total did we spend in 2011?"
response = query_router_engine.query(query_str)
print(str(response))
# display_response(response)

In 2011, we spent a total of $1,534,557.32.


In [15]:
data.loc[data.BscStartDate.str.startswith("2011")].Cost.sum()

1534557.3200000003

# Evaluating query and prompts

In [20]:
from llama_index.evaluation.guideline import DEFAULT_GUIDELINES
from llama_index.evaluation import GuidelineEvaluator
# Evaluating query and prompts
# from llama_index.evaluation.guideline import DEFAULT_GUIDELINES
# from llama_index.evaluation import GuidelineEvaluator
from llama_index.response.schema import Response
from llama_index.indices.query.query_transform.feedback_transform import (
    FeedbackQueryTransformation,)

# Guideline eval
guideline_eval = GuidelineEvaluator(
    guidelines=DEFAULT_GUIDELINES
    + "\nThe response should not be overly long.\n"
    "The response should try to summarize where possible.\n"
    "First, answer the question\n"
    "Second provide the reason, why you choose that answer.\n"
)  # just for example

typed_response = (
    retry_guideline_response if isinstance(retry_guideline_response, Response) else retry_guideline_response.get_response()
)
eval = guideline_eval.evaluate_response(query_str, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")

feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query_str, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")

Guideline eval evaluation result: The response fully answers the query by providing the specific amount spent in 2011, which is $1,534,557.32. It is not vague or ambiguous and uses statistics to provide a precise answer. The response is not overly long and effectively summarizes the information. Overall, it meets all the guidelines.
Transformed query: Here is a previous bad answer.
In 2011, we spent a total of $1,534,557.32.
Here is some feedback from the evaluator about the response given.
The response fully answers the query by providing the specific amount spent in 2011, which is $1,534,557.32. It is not vague or ambiguous and uses statistics to provide a precise answer. The response is not overly long and effectively summarizes the information. Overall, it meets all the guidelines.
Now answer the question.
How much money did we spend in total during the year 2011?


In [14]:
from typing import Tuple, Any
from llama_index.response.schema import Response
from llama_index.indices.query.query_transform.feedback_transform import FeedbackQueryTransformation
from llama_index.evaluation.guideline import DEFAULT_GUIDELINES
from llama_index.evaluation import GuidelineEvaluator

def evaluate_and_transform_query(
    query_str: str,
    retry_response: Response,  # Type hint as `Any` since it could be a `Response` or another type with a `.get_response()` method.
    DEFAULT_GUIDELINES: str
) -> Tuple[GuidelineEvaluator, FeedbackQueryTransformation]:
    """
    Evaluates a query response against a set of guidelines and transforms the query based on the feedback.

    Args:
        query_str (str): The original query string.
        retry_guideline_response (Any): The initial response to evaluate, can be a `Response` object or another type with a `get_response()` method.
        DEFAULT_GUIDELINES (str): The default guidelines string to be used for evaluation.

    Returns:
        Tuple[str, str]: A tuple containing the guideline evaluation feedback and the transformed query string.
    """
    # Initialize the guideline evaluator with additional guidelines.
    guideline_eval = GuidelineEvaluator(
        guidelines=DEFAULT_GUIDELINES
        + "\nThe response should not be overly long.\n"
          "The response should try to summarize where possible.\n"
          "First, answer the question\n"
          "Second provide the reason, why you choose that answer.\n"
    )

    # Get the typed response based on the type of `retry_guideline_response`.
    typed_response = (
        retry_response if isinstance(retry_response, Response) else retry_response.get_response()
    )

    # Evaluate the response against the guidelines.
    eval = guideline_eval.evaluate_response(query_str, typed_response)
    print(f"Guideline eval evaluation result: {eval.feedback}")

    # Transform the query based on feedback.
    feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
    transformed_query = feedback_query_transform.run(query_str, {"evaluation": eval})
    print(f"Transformed query: {transformed_query.query_str}")

    # Return the feedback and the transformed query string.
    return eval.feedback, transformed_query.query_str


In [15]:
feedback, transformed_query = evaluate_and_transform_query(query_str, response, DEFAULT_GUIDELINES)

Guideline eval evaluation result: The response fully answers the query by providing the specific amount spent in 2011, which is $1,534,557.32. It is not vague or ambiguous and uses statistics to provide a clear answer. The response is not overly long and summarizes the information effectively. Overall, it meets all the guidelines.
Transformed query: Here is a previous bad answer.
In 2011, we spent a total of $1,534,557.32.
Here is some feedback from the evaluator about the response given.
The response fully answers the query by providing the specific amount spent in 2011, which is $1,534,557.32. It is not vague or ambiguous and uses statistics to provide a clear answer. The response is not overly long and summarizes the information effectively. Overall, it meets all the guidelines.
Now answer the question.
How much money did we spend in total during the year 2011?


In [16]:
print(transformed_query)

Here is a previous bad answer.
In 2011, we spent a total of $1,534,557.32.
Here is some feedback from the evaluator about the response given.
The response fully answers the query by providing the specific amount spent in 2011, which is $1,534,557.32. It is not vague or ambiguous and uses statistics to provide a clear answer. The response is not overly long and summarizes the information effectively. Overall, it meets all the guidelines.
Now answer the question.
How much money did we spend in total during the year 2011?


In [17]:
print(feedback)

The response fully answers the query by providing the specific amount spent in 2011, which is $1,534,557.32. It is not vague or ambiguous and uses statistics to provide a clear answer. The response is not overly long and summarizes the information effectively. Overall, it meets all the guidelines.
